### SVM

In [9]:
import os
import json
import numpy as np
import cv2
from skimage.feature import hog
import base64

from sklearn.model_selection import cross_val_score

In [30]:
def im_transformation(im0):   
    # hog_feature = cv2.HOGDescriptor().compute(im0)
    fd = hog(im0, pixels_per_cell=(8, 8))
    
    return fd

In [48]:
def im_transformation2(im0):    
    # hog_feature = cv2.HOGDescriptor().compute(im0)
    fd = hog(im0, pixels_per_cell=(8, 8), block_norm='L2-Hys')
    
    return fd

In [36]:
def read_train(folder, im_fun, exceptions = {}):
    pathJson = folder
    jsonFiles = [fJson for fJson in os.listdir(pathJson) if fJson.endswith('.json')]

    bufferData = []
    labels = []

    for fJ in jsonFiles:
        with open(os.path.join(pathJson, fJ)) as infile:
            data = json.load(infile)
            
        if fJ in exceptions:
            label = exceptions[fJ]
        else: 
            for item in data["shapes"]:
                label = item["label"]
                label = label.upper()  # Convertir a mayúsculas
                label = ''.join(filter(str.isalpha, label))  # Eliminar números
                label = label.replace('JPG', '')  # Eliminar la extensión '.'

        imdata = base64.b64decode(data["imageData"])
        npimg = np.frombuffer(imdata, dtype=np.uint8)
        im0 = cv2.imdecode(npimg, 0)  # Lo pasamos a blanco y negro
        im0 = cv2.resize(im0, (150, 150))
        x = im_fun(im0)

        bufferData.append(x)
        labels.append(label)

        # Añadimos una imagen más
        # Girar la imagen, procesar y agregar
        # x = im_fun(im0)
        
        # bufferData.append(x)
        # labels.append(label)  # Asumiendo que la etiqueta es la misma para la imagen girada
            
    X = np.array(bufferData)
    Y = np.array(labels)
    return X, Y

In [49]:
manually_mod = {'0321.json': 'LF', '0322.json': 'LB', '0323.json': 'RB', '0324.json': 'RF', '0292.json': 'LB', '0306.json': 'RF', '0334.json': 'LF', '0335.json': 'LB',  '0333.json': 'RF', '0277.json': 'RF', '0278.json': 'RB', '0336.json': 'RF'}

X, Y = read_train("./HANDS", im_transformation2, manually_mod)

In [50]:
print(X.shape, Y.shape)

(398, 20736) (398,)


In [51]:
from sklearn.svm import SVC


# Crear el modelo
model = SVC(kernel="poly")

# Realizar la validación cruzada
scores = cross_val_score(model, X, Y, cv=5)  # cv es el número de divisiones/folds

# Mostrar los resultados
print(f"Exactitud en cada fold de la validación cruzada: {scores}")
print(f"Exactitud media: {scores.mean()}")


Exactitud en cada fold de la validación cruzada: [0.625      0.5125     0.75       0.60759494 0.4556962 ]
Exactitud media: 0.5901582278481012


Ahora vamos a probar el mismo modelo pero entrenandolo con imágenes giradas.

In [37]:
manually_mod = {'0321.json': 'LF', '0322.json': 'LB', '0323.json': 'RB', '0324.json': 'RF', '0292.json': 'LB', '0306.json': 'RF', '0334.json': 'LF', '0335.json': 'LB',  '0333.json': 'RF', '0277.json': 'RF', '0278.json': 'RB', '0336.json': 'RF'}

X, Y = read_train("./HANDS", lambda x: x, manually_mod) # queremos tener las imágenes sin formatear
print(X.shape, Y.shape)

(398, 150, 150) (398,)


In [43]:
def augment_data(X, Y):
    augmented_X = []
    augmented_Y = []

    for x, y in zip(X, Y):
        # Añadir la imagen original y su etiqueta
        x_p = im_transformation(x)
        augmented_X.append(x_p)
        augmented_Y.append(y)
        # Girar la imagen
        im_rotated = cv2.rotate(x, cv2.ROTATE_90_COUNTERCLOCKWISE)
        x_flipped = im_transformation(im_rotated)

        # Añadir la imagen girada y su etiqueta
        augmented_X.append(x_flipped)
        augmented_Y.append(y)

    
    return np.array(augmented_X), np.array(augmented_Y)
    

In [47]:
from sklearn.model_selection import KFold

# Configuración de la validación cruzada
kf = KFold(n_splits=5)

model = SVC(kernel="poly")

for train_index, test_index in kf.split(X):
    # Dividir los datos
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    # Aplicar la aumentación de datos solo al conjunto de entrenamiento
    X_train_augmented, Y_train_augmented = augment_data(X_train, Y_train)

    # Entrenar el modelo con los datos aumentados
    model.fit(X_train_augmented, Y_train_augmented)
    
    # Obtener HOGS de test
    X_test = np.array([im_transformation(row) for row in X_test])
    
    # Evaluar el modelo
    score = model.score(X_test, Y_test)
    print("Exactitud en el conjunto de prueba:", score)


(80, 20736) (80,)
Exactitud en el conjunto de prueba: 0.6
(80, 20736) (80,)
Exactitud en el conjunto de prueba: 0.575
(80, 20736) (80,)
Exactitud en el conjunto de prueba: 0.6875
(79, 20736) (79,)
Exactitud en el conjunto de prueba: 0.7215189873417721
(79, 20736) (79,)
Exactitud en el conjunto de prueba: 0.4810126582278481


Media: 0.613 > 0.59

Ha mejorado un poco el modelo anterior. Por lo tanto nos lo vamos a guardar.